## 任务5：使用网格搜索法对5个模型进行调优（调参时采用五折交叉验证的方式），并进行模型评估。

### 数据载入

In [1]:
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

# 数据集预览
data = pd.read_csv("./data.csv",encoding='gbk')
# 去重
data.drop_duplicates(inplace=True)
print(data.shape)
data.head()

(4754, 90)


Unnamed: 0   custid                          trade_no bank_card_no  \
0           5  2791858  20180507115231274000000023057383          卡号1   
1          10   534047  20180507121002192000000023073000          卡号1   
2          12  2849787  20180507125159718000000023114911          卡号1   
3          13  1809708  20180507121358683000000388283484          卡号1   
4          14  2499829  20180507115448545000000388205844          卡号1   

   low_volume_percent  middle_volume_percent  \
0                0.01                   0.99   
1                0.02                   0.94   
2                0.04                   0.96   
3                0.00                   0.96   
4                0.01                   0.99   

   take_amount_in_later_12_month_highest  trans_amount_increase_rate_lately  \
0                                      0                               0.90   
1                                   2000                               1.28   
2                                      0                               1.00   
3                                   2000                               0.13   
4                                      0                               0.46   

   trans_activity_month  trans_activity_day  ...  loans_max_limit  \
0                  0.55               0.313  ...           2900.0   
1                  1.00               0.458  ...           3500.0   
2                  1.00               0.114  ...           1600.0   
3                  0.57               0.777  ...           3200.0   
4                  1.00               0.175  ...           2300.0   

   loans_avg_limit  consfin_credit_limit  consfin_credibility  \
0           1688.0                1200.0                 75.0   
1           1758.0               15100.0                 80.0   
2           1250.0                4200.0                 87.0   
3           1541.0               16300.0                 80.0   
4           1630.0                8300.0                 79.0   

   consfin_org_count_current  consfin_product_count  consfin_max_limit  \
0                        1.0                    2.0             1200.0   
1                        5.0                    6.0            22800.0   
2                        1.0                    1.0             4200.0   
3                        5.0                    5.0            30000.0   
4                        2.0                    2.0             8400.0   

   consfin_avg_limit  latest_query_day  loans_latest_day  
0             1200.0              12.0              18.0  
1             9360.0               4.0               2.0  
2             4200.0               2.0               6.0  
3            12180.0               2.0               4.0  
4             8250.0              22.0             120.0  

[5 rows x 90 columns]

In [3]:
# 观测正负样本是否均衡
y = data.status
y.value_counts()

0    3561
1    1193
Name: status, dtype: int64

In [6]:
#载入特征
import numpy as np
X = np.loadtxt('X.txt')
X.shape

(4754, 80)

### 数据集划分

In [8]:
# 划分训练集测试集
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=2018)

# 特征归一化
std = StandardScaler()
X_train = std.fit_transform(X_train)
X_test = std.transform(X_test)

### 模型评估

In [9]:
from sklearn.metrics import accuracy_score, roc_auc_score

def model_metrics(clf, X_train, X_test, y_train, y_test):
    # 预测
    y_train_pred = clf.predict(X_train)
    y_test_pred = clf.predict(X_test)
    
    y_train_proba = clf.predict_proba(X_train)[:,1]
    y_test_proba = clf.predict_proba(X_test)[:,1]
    
    # 准确率
    print('[准确率]', end = ' ')
    print('训练集：', '%.4f'%accuracy_score(y_train, y_train_pred), end = ' ')
    print('测试集：', '%.4f'%accuracy_score(y_test, y_test_pred))
    
    # auc取值：用roc_auc_score或auc
    print('[auc值]', end = ' ')
    print('训练集：', '%.4f'%roc_auc_score(y_train, y_train_proba), end = ' ')
    print('测试集：', '%.4f'%roc_auc_score(y_test, y_test_proba))

In [10]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from xgboost.sklearn import XGBClassifier
from lightgbm.sklearn import LGBMClassifier

### LR模型

In [11]:
lr = LogisticRegression()
param = {'C': [1e-3,0.01,0.1,1,10,100,1e3], 'penalty':['l1', 'l2']}

gsearch = GridSearchCV(lr, param_grid = param,scoring ='roc_auc', cv=5)
gsearch.fit(X_train, y_train)

print('最佳参数：',gsearch.best_params_)
print('训练集的最佳分数：', gsearch.best_score_)
print('测试集的最佳分数：', gsearch.score(X_test, y_test))

最佳参数： {'C': 0.01, 'penalty': 'l2'}
训练集的最佳分数： 0.7922462174492412
测试集的最佳分数： 0.7746366832545668


In [41]:
lr = LogisticRegression(C = 0.01, penalty = 'l2')
lr.fit(X_train, y_train)
model_metrics(lr, X_train, X_test, y_train, y_test)

[准确率] 训练集： 0.8019 测试集： 0.7870
[auc值] 训练集： 0.8145 测试集： 0.7746


### SVM模型
    
    调参范围可设为'gamma':[0.001,0.01,0.1,1,10,100], 'C':[0.001,0.01,0.1,1,10,100]}。鉴于时间原因, 下面网格搜索时选用较小区间。

In [13]:
# 线性SVM
svm_linear = svm.SVC(kernel = 'linear', probability=True)
param = {'C':[0.01,0.1,1]}
gsearch = GridSearchCV(svm_linear, param_grid = param,scoring ='roc_auc', cv=5)
gsearch.fit(X_train, y_train)

print('最佳参数：',gsearch.best_params_)
print('训练集的最佳分数：', gsearch.best_score_)
print('测试集的最佳分数：', gsearch.score(X_test, y_test))

最佳参数： {'C': 0.01}
训练集的最佳分数： 0.796732239847577
测试集的最佳分数： 0.7810423252271708


In [14]:
svm_linear = svm.SVC(C = 0.01, kernel = 'linear', probability=True)
svm_linear.fit(X_train, y_train)
model_metrics(svm_linear, X_train, X_test, y_train, y_test)

[准确率] 训练集： 0.8001 测试集： 0.7765
[auc值] 训练集： 0.8155 测试集： 0.7810


In [15]:
# 多项式SVM
svm_poly = svm.SVC(kernel = 'poly', probability=True)
param = {'C':[0.01,0.1,1]}
gsearch = GridSearchCV(svm_poly, param_grid = param,scoring ='roc_auc', cv=5)
gsearch.fit(X_train, y_train)

print('最佳参数：',gsearch.best_params_)
print('训练集的最佳分数：', gsearch.best_score_)
print('测试集的最佳分数：', gsearch.score(X_test, y_test))

最佳参数： {'C': 0.01}
训练集的最佳分数： 0.7464537856311786
测试集的最佳分数： 0.7314820610726842


In [16]:
svm_poly =  svm.SVC(C = 0.01, kernel = 'poly', probability=True)
svm_poly.fit(X_train, y_train)
model_metrics(svm_poly, X_train, X_test, y_train, y_test)

[准确率] 训练集： 0.7571 测试集： 0.7498
[auc值] 训练集： 0.8547 测试集： 0.7315


In [18]:
# 高斯SVM
svm_rbf = svm.SVC(probability=True)
param = {'gamma':[0.01,0.1,1,10], 
         'C':[0.01,0.1,1]}
gsearch = GridSearchCV(svm_poly, param_grid = param,scoring ='roc_auc', cv=5)
gsearch.fit(X_train, y_train)

print('最佳参数：',gsearch.best_params_)
print('训练集的最佳分数：', gsearch.best_score_)
print('测试集的最佳分数：', gsearch.score(X_test, y_test))

最佳参数： {'C': 0.01, 'gamma': 0.01}
训练集的最佳分数： 0.7488403772207765
测试集的最佳分数： 0.7342049805431232


In [19]:
svm_rbf =  svm.SVC(gamma = 0.01, C =0.01 , probability=True)
svm_rbf.fit(X_train, y_train)
model_metrics(svm_rbf, X_train, X_test, y_train, y_test)


[准确率] 训练集： 0.7493 测试集： 0.7484
[auc值] 训练集： 0.8454 测试集： 0.7687


In [20]:
# sigmoid - SVM
svm_sigmoid = svm.SVC(kernel = 'sigmoid',probability=True)
param = {'C':[0.01,0.1,1]}
gsearch = GridSearchCV(svm_sigmoid, param_grid = param,scoring ='roc_auc', cv=5)
gsearch.fit(X_train, y_train)

print('最佳参数：',gsearch.best_params_)
print('训练集的最佳分数：', gsearch.best_score_)
print('测试集的最佳分数：', gsearch.score(X_test, y_test))

最佳参数： {'C': 0.01}
训练集的最佳分数： 0.7781922993659
测试集的最佳分数： 0.7619192930842018


In [21]:
svm_sigmoid =  svm.SVC(C = 0.01, kernel = 'sigmoid',probability=True)
svm_sigmoid.fit(X_train, y_train)
model_metrics(svm_sigmoid, X_train, X_test, y_train, y_test)

[准确率] 训练集： 0.7493 测试集： 0.7484
[auc值] 训练集： 0.7746 测试集： 0.7619


### 决策树

1）首先对决策树最大深度max_depth和内部节点再划分所需最小样本数min_samples_split进行网格搜索。

In [22]:
param = {'max_depth':range(3,14,2), 'min_samples_split':range(100,801,200)}
gsearch = GridSearchCV(DecisionTreeClassifier(max_depth=8,min_samples_split=300,min_samples_leaf=20, max_features='sqrt',random_state =2333),
                       param_grid = param,scoring ='roc_auc', cv=5)

gsearch.fit(X_train, y_train)
# gsearch.grid_scores_, 
gsearch.best_params_, gsearch.best_score_

({'max_depth': 5, 'min_samples_split': 300}, 0.686150110652568)

2）对内部节点再划分所需最小样本数min_samples_split和叶子节点最少样本数min_samples_leaf一起调参。

In [23]:
param = {'min_samples_split':range(50,1000,100), 'min_samples_leaf':range(60,101,10)}
gsearch = GridSearchCV(DecisionTreeClassifier(max_depth=11,min_samples_split=100,min_samples_leaf=20, max_features='sqrt', random_state =2333),
                       param_grid = param,scoring ='roc_auc', cv=5)

gsearch.fit(X_train, y_train)
# gsearch.grid_scores_, 
gsearch.best_params_, gsearch.best_score_

({'min_samples_leaf': 90, 'min_samples_split': 50}, 0.7013758256451866)

3）再对最大特征数max_features进行网格搜索

In [24]:
param = {'max_features':range(7,20,2)}
gsearch = GridSearchCV(DecisionTreeClassifier(max_depth=11,min_samples_split=550,min_samples_leaf=80, max_features='sqrt', random_state =2333),
                       param_grid = param,scoring ='roc_auc', cv=5)

gsearch.fit(X_train, y_train)
# gsearch.grid_scores_, 
gsearch.best_params_, gsearch.best_score_

({'max_features': 17}, 0.7112403211564169)

In [42]:
dt = DecisionTreeClassifier(max_depth=5,min_samples_split=50,min_samples_leaf=90,max_features=17, random_state =2018)
dt.fit(X_train, y_train)
model_metrics(dt, X_train, X_test, y_train, y_test)

[准确率] 训练集： 0.7821 测试集： 0.7484
[auc值] 训练集： 0.7517 测试集： 0.7100


### XGBoost模型

1、max_depth = 5 :这个参数的取值最好在3-10之间。我选的起始值为5，但是你也可以选择其它的值。起始值在4-6之间都是不错的选择。
2、min_child_weight = 1:在这里选了一个比较小的值，因为这是一个极不平衡的分类问题。因此，某些叶子节点下的值会比较小。
3、gamma = 0: 起始值也可以选其它比较小的值，在0.1到0.2之间就可以。这个参数后继也是要调整的。
4、subsample, colsample_bytree = 0.8: 这个是最常见的初始值了。典型值的范围在0.5-0.9之间。

首先看一下默认参数的结果

In [43]:
import warnings
warnings.filterwarnings("ignore")

xgb0 = XGBClassifier()
xgb0.fit(X_train, y_train)

model_metrics(xgb0, X_train, X_test, y_train, y_test)

[准确率] 训练集： 0.8539 测试集： 0.7877
[auc值] 训练集： 0.9161 测试集： 0.7741


1) 首先从步长(learning rate)和迭代次数(n_estimators)入手。
开始选择一个较小的步长来网格搜索最好的迭代次数。这里，我们将步长初始值设置为0.1, 对于迭代次数进行网格搜索。

In [44]:
param_test = {'n_estimators':range(20,200,20)}
gsearch = GridSearchCV(estimator = XGBClassifier(learning_rate =0.1, n_estimators=140, max_depth=5, 
                                                  min_child_weight=1, gamma=0, subsample=0.8, 
                                                  colsample_bytree=0.8, objective= 'binary:logistic', 
                                                  nthread=4,scale_pos_weight=1, seed=27), 
                        param_grid = param_test, scoring='roc_auc',n_jobs=4,iid=False, cv=5)

gsearch.fit(X_train, y_train)
# gsearch.grid_scores_, 
gsearch.best_params_, gsearch.best_score_

({'n_estimators': 40}, 0.7976610009502891)

2) max_depth 和 min_child_weight 参数调优

In [45]:
param_test = {'max_depth':range(3,10,2), 'min_child_weight':range(1,12,2)}

gsearch = GridSearchCV(estimator = XGBClassifier(learning_rate =0.1, n_estimators=40, max_depth=5, 
                                                  min_child_weight=1, gamma=0, subsample=0.8, 
                                                  colsample_bytree=0.8, objective= 'binary:logistic', 
                                                  nthread=4,scale_pos_weight=1, seed=27), 
                        param_grid = param_test, scoring='roc_auc',n_jobs=4,iid=False, cv=5)

gsearch.fit(X_train, y_train)
# gsearch.grid_scores_, 
gsearch.best_params_, gsearch.best_score_

({'max_depth': 3, 'min_child_weight': 1}, 0.8023180736067406)

可以看出理想的max_depth值为5，理想的min_child_weight值为3。在这个值附近我们可以再进一步调整，来找出理想值。

In [46]:
param_test = {'max_depth':[3,4,5], 'min_child_weight':[3,4,5]}

gsearch = GridSearchCV(estimator = XGBClassifier(learning_rate =0.1, n_estimators=40, max_depth=3, 
                                                  min_child_weight=1, gamma=0, subsample=0.8, 
                                                  colsample_bytree=0.8, objective= 'binary:logistic', 
                                                  nthread=4,scale_pos_weight=1, seed=27), 
                        param_grid = param_test, scoring='roc_auc',n_jobs=4,iid=False, cv=5)

gsearch.fit(X_train, y_train)
# gsearch.grid_scores_, 
gsearch.best_params_, gsearch.best_score_

({'max_depth': 3, 'min_child_weight': 3}, 0.8016561809606216)

3) gamma参数调优

In [47]:
param_test = {'gamma':[i/10 for i in range(1,6)]}

gsearch = GridSearchCV(estimator = XGBClassifier(learning_rate =0.1, n_estimators=40, max_depth=3, 
                                                  min_child_weight=3, gamma=0, subsample=0.8, 
                                                  colsample_bytree=0.8, objective= 'binary:logistic', 
                                                  nthread=4,scale_pos_weight=1, seed=27), 
                        param_grid = param_test, scoring='roc_auc',n_jobs=4,iid=False, cv=5)

gsearch.fit(X_train, y_train)
# gsearch.grid_scores_, 
gsearch.best_params_, gsearch.best_score_

({'gamma': 0.1}, 0.8016561809606216)

4）调整subsample 和 colsample_bytree 参数

In [48]:
param_test = {'subsample':[i/10 for i in range(5,10)], 'colsample_bytree':[i/10 for i in range(5,10)]}

gsearch = GridSearchCV(estimator = XGBClassifier(learning_rate =0.1, n_estimators=40, max_depth=3, 
                                                  min_child_weight=3, gamma=0.1, subsample=0.8, 
                                                  colsample_bytree=0.8, objective= 'binary:logistic', 
                                                  nthread=4,scale_pos_weight=1, seed=27), 
                        param_grid = param_test, scoring='roc_auc',n_jobs=4,iid=False, cv=5)

gsearch.fit(X_train, y_train)
# gsearch.grid_scores_, 
gsearch.best_params_, gsearch.best_score_

({'colsample_bytree': 0.5, 'subsample': 0.9}, 0.8029258723508983)

从这里可以看出来，subsample理想取值0.8, colsample_bytree理想取值都是0.6。现在，我们以0.05为步长，在这个值附近尝试取值。

In [49]:
param_test = { 'subsample':[i/100 for i in range(85,101,5)], 'colsample_bytree':[i/100 for i in range(85,101,5)]}

gsearch = GridSearchCV(estimator = XGBClassifier(learning_rate =0.1, n_estimators=40, max_depth=3, 
                                                  min_child_weight=3, gamma=0.1, subsample=0.9, 
                                                  colsample_bytree=0.5, objective= 'binary:logistic', 
                                                  nthread=4,scale_pos_weight=1, seed=27), 
                        param_grid = param_test, scoring='roc_auc',n_jobs=4,iid=False, cv=5)

gsearch.fit(X_train, y_train)
# gsearch.grid_scores_, 
gsearch.best_params_, gsearch.best_score_

({'colsample_bytree': 0.85, 'subsample': 0.9}, 0.8026923203352562)

5）正则化参数调优
        
        'reg_lambda': [0.2, 0.4, 0.6, 0.8, 1]

In [50]:
param_test = {'reg_alpha':[1e-5, 1e-2, 0.1, 0, 1, 100]}

gsearch = GridSearchCV(estimator = XGBClassifier(learning_rate =0.1, n_estimators=40, max_depth=3, 
                                                  min_child_weight=3, gamma=0.1, subsample=0.9, 
                                                  colsample_bytree=0.85, objective= 'binary:logistic', 
                                                  nthread=4,scale_pos_weight=1, seed=27), 
                        param_grid = param_test, scoring='roc_auc',n_jobs=4,iid=False, cv=5)

gsearch.fit(X_train, y_train)
# gsearch.grid_scores_, 
gsearch.best_params_, gsearch.best_score_

({'reg_alpha': 1e-05}, 0.8026923203352562)

6）继续从1）循环调整, 此处省略步骤。

7）回到第1）步，降低学习速率, 调整迭代次数

In [51]:
param_test = {'n_estimators':range(20,200,20)}

gsearch = GridSearchCV(estimator = XGBClassifier(learning_rate =0.1, n_estimators=40, max_depth=3, 
                                                  min_child_weight=3, gamma=0.1, subsample=0.9, 
                                                  colsample_bytree=0.85, reg_alpha=1e-05, objective= 'binary:logistic', 
                                                  nthread=4,scale_pos_weight=1, seed=27), 
                        param_grid = param_test, scoring='roc_auc',n_jobs=4,iid=False, cv=5)

gsearch.fit(X_train, y_train)
# gsearch.grid_scores_, 
gsearch.best_params_, gsearch.best_score_

({'n_estimators': 60}, 0.8038581704914713)

与1）结果相比, 较好。所以, 选用此处学习速率较小时的结果。继续从1）循环调整, 此处不再调整。

In [52]:
xgb = XGBClassifier(learning_rate =0.1, n_estimators=60, max_depth=3, min_child_weight=3, 
                    gamma=0.1, subsample=0.9, colsample_bytree=0.85, reg_alpha=1e-05, 
                    objective= 'binary:logistic', nthread=4,scale_pos_weight=1, seed=27)
xgb.fit(X_train, y_train)
model_metrics(xgb, X_train, X_test, y_train, y_test)

[准确率] 训练集： 0.8299 测试集： 0.7905
[auc值] 训练集： 0.8827 测试集： 0.7746


### LightGBM模型

In [53]:
# 首先看一下默认参数的结果
lgb0 = LGBMClassifier()
lgb0.fit(X_train, y_train)

model_metrics(lgb0, X_train, X_test, y_train, y_test)

[准确率] 训练集： 0.9940 测试集： 0.7673
[auc值] 训练集： 1.0000 测试集： 0.7659


In [54]:
param_test = {'n_estimators':range(20,200,20)}
gsearch = GridSearchCV(estimator = LGBMClassifier(learning_rate =0.1, n_estimators=60, max_depth=3, 
                                                  min_child_weight=11, gamma=0.1, subsample=0.5, 
                                                  colsample_bytree=0.8, reg_alpha = 1e-5,
                                                  nthread=4,scale_pos_weight=1, seed=27), 
                        param_grid = param_test, scoring='roc_auc',n_jobs=4,iid=False, cv=5)

gsearch.fit(X_train, y_train)
# gsearch.grid_scores_, 

gsearch.best_params_, gsearch.best_score_

({'n_estimators': 40}, 0.8017769700633165)

In [55]:
lgb = LGBMClassifier(learning_rate =0.1, n_estimators=40, max_depth=3, min_child_weight=11, 
                    gamma=0.1, subsample=0.5, colsample_bytree=0.8, reg_alpha=1e-5, 
                    nthread=4,scale_pos_weight=1, seed=27)
lgb.fit(X_train, y_train)
model_metrics(lgb, X_train, X_test, y_train, y_test)

[准确率] 训练集： 0.8239 测试集： 0.7933
[auc值] 训练集： 0.8619 测试集： 0.7759


|模型|参数|准确率|auc值|
|:---|:---|:---|:---|
|LR|LogisticRegression(C = 0.01, penalty = 'l2')|训练集： 0.8013 测试集： 0.7877|训练集： 0.8136 测试集： 0.7786|
|svm_linear|svm.SVC(C = 0.01, kernel = 'linear', probability=True)|训练集： 0.8001 测试集： 0.7765|训练集： 0.8155 测试集： 0.7810|
|svm_poly|svm.SVC(C = 0.01, kernel = 'poly', probability=True)|训练集： 0.7538 测试集： 0.7547| 训练集： 0.8690 测试集： 0.7260|
|svm_rbf|svm.SVC(gamma = 0.01, C =0.01 , probability=True)|训练集： 0.7493 测试集： 0.7484|训练集： 0.8454 测试集： 0.7687|
|svm_sigmoid|svm.SVC(C = 0.01, kernel = 'sigmoid',probability=True)|训练集： 0.7493 测试集： 0.7484|训练集： 0.7746 测试集： 0.7619|
|决策树|DecisionTreeClassifier(max_depth=5,min_samples_split=300,min_samples_leaf=90,max_features=17)|训练集： 0.7650 测试集： 0.7449|训练集： 0.7509 测试集： 0.6879|
|XGBoost|XGBClassifier(learning_rate =0.01, n_estimators=180, max_depth=3, min_child_weight=5,gamma=0.4, subsample=0.5, colsample_bytree=0.9, reg_alpha=1, objective= 'binary:logistic',nthread=4,scale_pos_weight=1, seed=27)|训练集： 0.8142 测试集： 0.7835|训练集： 0.8318 测试集： 0.7680|
|LightGBM|LGBMClassifier(learning_rate =0.1, n_estimators=40, max_depth=3, min_child_weight=11,gamma=0.1, subsample=0.5, colsample_bytree=0.8, reg_alpha=1e-5, nthread=4,scale_pos_weight=1, seed=27)|训练集： 0.8308 测试集： 0.7926|训练集： 0.8791 测试集： 0.7756|